In [2]:
import pyodbc
import sqlite3
import pandas as pd

export_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=(localdb)\\MSSQLLocalDB;'
    'DATABASE=projectdatawarehouse;'
    'Trusted_Connection=yes;'
)

export_cursor = export_conn.cursor()

import_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=(localdb)\\MSSQLLocalDB;'
    'DATABASE=projectsourcedatamodel;'
    'Trusted_Connection=yes;'
)

import_cursor = import_conn.cursor()

In [25]:
def clear_tables():
    tables = [
        'Fact_EmployeePerformance',
        'Fact_Sales',
        'Fact_Purchase',
        'Dim_Employee',
        'Dim_Location',
        'Dim_DateTime',
        'Dim_Product',
        'Dim_Vendor',
        'Dim_Customer'
    ]

    for table in tables:
        export_cursor.execute(f"DELETE FROM {table}")
        export_conn.commit()

    print("Alle tabellen zijn geleegd.")

clear_tables()

Alle tabellen zijn geleegd.


In [26]:
def move_dimcustomer(import_conn, export_conn):
    # Data ophalen uit de verschillende tabellen
    customer = pd.read_sql_query("SELECT CustomerID, Fname, Lname, Phone, Address, City, Region, State, Country FROM Customer", import_conn)
    customer_customer_demo = pd.read_sql_query("SELECT CustomerID, CustomerTypeID FROM CustomerCustomerDemo", import_conn)
    customer_demographics = pd.read_sql_query("SELECT CustomerTypeID, CustomerDesc FROM CustomerDemographics", import_conn)
    
    # Data mergen met pandas
    merged_df = customer.merge(customer_customer_demo, on='CustomerID', how='left')
    merged_df = merged_df.merge(customer_demographics, on='CustomerTypeID', how='left')
    
    # Kolommen transformeren
    merged_df['FullName'] = merged_df['Fname'] + ' ' + merged_df['Lname']
    merged_df['Region'] = merged_df['Region'].combine_first(merged_df['State'])  # Gebruik Region, fallback op State
    
    # Selecteer alleen de benodigde kolommen
    final_df = merged_df[[
        'CustomerID', 
        'FullName', 
        'CustomerDesc',  # Dit is de CustomerType die we willen
        'Phone', 
        'Address', 
        'City', 
        'Region', 
        'Country'
    ]].rename(columns={'CustomerDesc': 'CustomerType'})
    
    # Data naar doeltablet schrijven
    cursor = export_conn.cursor()
    
    for index, row in final_df.iterrows():
        try:
            query = '''
                INSERT INTO Dim_Customer 
                (CustomerID, FullName, CustomerType, PhoneNumber, Address, City, Region, Country) 
                SELECT ?, ?, ?, ?, ?, ?, ?, ?
                WHERE NOT EXISTS (
                    SELECT 1 FROM Dim_Customer WHERE CustomerID = ?
                )
            '''
            cursor.execute(query, tuple(row.fillna('')) + (row['CustomerID'],))
        except Exception as e:
            print(f"Fout bij invoegen customer {row['CustomerID']}: {e}")
    
    export_conn.commit()
    print(f"{len(final_df)} customers verwerkt in Dim_Customer")

# Roep de functie aan
move_dimcustomer(import_conn, export_conn)

C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\324957438.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer = pd.read_sql_query("SELECT CustomerID, Fname, Lname, Phone, Address, City, Region, State, Country FROM Customer", import_conn)
C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\324957438.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_customer_demo = pd.read_sql_query("SELECT CustomerID, CustomerTypeID FROM CustomerCustomerDemo", import_conn)
C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\324957438.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAP

217 customers verwerkt in Dim_Customer


In [27]:
def move_dimemployee():
    import_cursor.execute("""
        SELECT 
            e.EmployeeID,
            e.FirstName + ' ' + e.LastName AS FullName,
            COALESCE(e.JobTitle, e.Title, 'Onbekend') AS JobTitle,
            d.dept_name AS Department,
            e.HireDate,
            e.BirthDate,
            NULL AS EmailAddress,
            NULL AS ManagerID  
        FROM Employee e
        LEFT JOIN Department d ON e.DepartmentID = d.dept_id
    """)
    
    employees = import_cursor.fetchall()
    
    for employee in employees:
        export_cursor.execute("""
            INSERT INTO Dim_Employee (
                EmployeeID,
                FullName,
                JobTitle,
                Department,
                HireDate,
                BirthDate,
                EmailAddress,
                ManagerID
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, employee)
    
    import_cursor.execute("""
        SELECT e.EmployeeID, e.manager_id
        FROM Employee e
        WHERE e.manager_id IS NOT NULL
    """)
    
    manager_pairs = import_cursor.fetchall()
    
    for emp_id, manager_id in manager_pairs:
        export_cursor.execute("""
            UPDATE Dim_Employee
            SET ManagerID = ?
            WHERE EmployeeID = ?
            AND EXISTS (SELECT 1 FROM Dim_Employee WHERE EmployeeID = ?)  
        """, (manager_id, emp_id, manager_id))
    
    export_conn.commit()
    print(f"{len(employees)} employees imported into Dim_Employee")

move_dimemployee()

351 employees imported into Dim_Employee


In [28]:
def move_dimproduct(import_conn, export_conn):
    # Data ophalen uit de verschillende tabellen
    product = pd.read_sql_query("""
        SELECT 
            ProductID, 
            Name AS ProductName,
            Color,
            StandardCost,
            ListPrice,
            ProductLine,
            Discontinued,
            CategoryID
        FROM Product
    """, import_conn)
    
    category = pd.read_sql_query("""
        SELECT 
            CategoryID, 
            CategoryName AS ProductCategoryName
        FROM Category
    """, import_conn)
    
    # Data mergen met pandas
    merged_df = product.merge(category, on='CategoryID', how='left')
    
    # Discontinued omzetten naar DiscontinuedDate (indien discontinued)
    merged_df['DiscontinuedDate'] = merged_df['Discontinued'].apply(
        lambda x: pd.Timestamp.today().date() if x == 1 else None
    )
    
    # Selecteer alleen de benodigde kolommen
    final_df = merged_df[[
        'ProductID', 
        'ProductName', 
        'ProductCategoryName', 
        'Color', 
        'StandardCost', 
        'ListPrice', 
        'ProductLine', 
        'DiscontinuedDate'
    ]]
    
    # Data naar doeltablet schrijven
    cursor = export_conn.cursor()
    
    for index, row in final_df.iterrows():
        try:
            query = '''
                INSERT INTO Dim_Product 
                (ProductID, ProductName, ProductCategoryName, Color, 
                 StandardCost, ListPrice, ProductLine, DiscontinuedDate) 
                SELECT ?, ?, ?, ?, ?, ?, ?, ?
                WHERE NOT EXISTS (
                    SELECT 1 FROM Dim_Product WHERE ProductID = ?
                )
            '''
            # Tuple van waarden + ProductID voor de WHERE NOT EXISTS check
            values = tuple(row.fillna('') if isinstance(row, pd.Series) else row) + (row['ProductID'],)
            cursor.execute(query, values)
        except Exception as e:
            print(f"Fout bij invoegen product {row['ProductID']}: {e}")
    
    export_conn.commit()
    print(f"{len(final_df)} products verwerkt in Dim_Product")

# Roep de functie aan
move_dimproduct(import_conn, export_conn)

C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\2999684223.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql_query("""
C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\2999684223.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  category = pd.read_sql_query("""


585 products verwerkt in Dim_Product


In [ ]:
import pandas as pd

def move_dimlocation(import_conn, export_conn):
    # Stap 1: Data ophalen uit de verschillende tabellen
    customer = pd.read_sql_query("SELECT CustomerID, Address AS StreetAddress, City, State FROM Customer", import_conn)
    state = pd.read_sql_query("SELECT State_id, State_name AS StateProvince, Country FROM State", import_conn)
    order = pd.read_sql_query("SELECT OrderID, CustomerID, EmployeeID FROM [Order]", import_conn)
    employee = pd.read_sql_query("SELECT EmployeeID, PostalCode FROM Employee", import_conn)
    emp_terr = pd.read_sql_query("SELECT EmployeeID, TerritoryID FROM EmployeeTerritories", import_conn)
    territories = pd.read_sql_query("SELECT TerritoryID, RegionID FROM Territories", import_conn)
    region = pd.read_sql_query("SELECT RegionID, RegionDescription AS Region FROM Region", import_conn)

    # Stap 2: Merges uitvoeren
    df = customer.merge(state, left_on='State', right_on='State_id', how='left') \
                 .merge(order, on='CustomerID', how='inner') \
                 .merge(employee, on='EmployeeID', how='inner') \
                 .merge(emp_terr, on='EmployeeID', how='inner') \
                 .merge(territories, on='TerritoryID', how='inner') \
                 .merge(region, on='RegionID', how='inner')

    # Stap 3: Selecteer en hernoem benodigde kolommen
    final_df = df[[
        'StreetAddress', 
        'City', 
        'StateProvince', 
        'Region', 
        'Country', 
        'PostalCode'
    ]].drop_duplicates()

    # Stap 4: Data invoegen in Dim_Location
    cursor = export_conn.cursor()

    for index, row in final_df.iterrows():
        try:
            query = '''
                INSERT INTO Dim_Location 
                (StreetAddress, City, StateProvince, Region, Country, PostalCode)
                SELECT ?, ?, ?, ?, ?, ?
                WHERE NOT EXISTS (
                    SELECT 1 FROM Dim_Location 
                    WHERE StreetAddress = ? AND City = ? AND StateProvince = ? 
                          AND Region = ? AND Country = ? AND PostalCode = ?
                )
            '''
            
            values = tuple(
                None if pd.isna(row[col]) else row[col]
                for col in ['StreetAddress', 'City', 'StateProvince', 'Region', 'Country', 'PostalCode'] * 2
            )
            cursor.execute(query, values)
        except Exception as e:
            print(f"Fout bij invoegen locatie (rij {index}): {e}")

    export_conn.commit()
    print(f"{len(final_df)} locaties verwerkt in Dim_Location")

# Roep de functie aan
move_dimlocation(import_conn, export_conn)


C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\923070079.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer = pd.read_sql_query("SELECT CustomerID, Address AS StreetAddress, City, State FROM Customer", import_conn)
C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\923070079.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  state = pd.read_sql_query("SELECT State_id, State_name AS StateProvince, Country FROM State", import_conn)
C:\Users\ibrah\AppData\Local\Temp\ipykernel_20572\923070079.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please

464 unieke locaties verwerkt in Dim_Location


In [ ]:

def move_feitsales(import_conn, export_conn):
    # Stap 1: Data ophalen uit de bron
    header = pd.read_sql_query("""
        SELECT 
            SalesOrderID,
            OrderDate,
            CustomerID,
            NULL AS LocationID,
            Freight,
            TaxAmt,
            TotalDue
        FROM Sales_SalesOrderHeader
    """, import_conn)

    detail = pd.read_sql_query("""
        SELECT 
            SalesOrderID,
            SalesOrderDetailID AS SalesID,
            ProductID,
            OrderQty AS Quantity,
            UnitPrice,
            UnitPriceDiscount,
            LineTotal
        FROM Sales_SalesOrderDetail
    """, import_conn)

    # Stap 2: Merge header en detail via SalesOrderID
    merged_df = detail.merge(header, on='SalesOrderID', how='inner')

    # Stap 3: Transformaties
    merged_df['DiscountAmount'] = merged_df['UnitPrice'] * merged_df['UnitPriceDiscount'] * merged_df['Quantity']
    merged_df['SalesAmount'] = merged_df['LineTotal'] - merged_df['DiscountAmount']
    merged_df['Profit'] = merged_df['TotalDue'] - merged_df['LineTotal'] / merged_df['TotalDue'] 

    # (Optioneel) Genereer SaleDateID via datum-dimensie mapping
    # Hier doen we gewoon een voorbeeld met jaarmaanddag als int
    merged_df['SaleDateID'] = pd.to_datetime(merged_df['OrderDate']).dt.strftime('%Y%m%d').astype(int)

    # Stap 4: Kolommen selecteren zoals in Fact_Sales
    final_df = merged_df[[
        'SalesID',
        'SaleDateID',
        'ProductID',
        'CustomerID',
        'LocationID',
        'Quantity',
        'UnitPrice',
        'DiscountAmount',
        'SalesAmount',
        'TaxAmt',
        'TotalDue',
        'Profit'
    ]]

    # Stap 5: Data invoegen in Fact_Sales
    cursor = export_conn.cursor()
    for index, row in final_df.iterrows():
        try:
            query = '''
                INSERT INTO Fact_Sales 
                (SalesID, SaleDateID, ProductID, CustomerID, LocationID,
                 Quantity, UnitPrice, DiscountAmount, SalesAmount, TaxAmount, TotalAmount, Profit)
                SELECT ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                WHERE NOT EXISTS (
                    SELECT 1 FROM Fact_Sales WHERE SalesID = ?
                )
            '''
            values = tuple(
                None if pd.isna(row[col]) else row[col]
                for col in final_df.columns
            ) + (row['SalesID'],)
            cursor.execute(query, values)
        except Exception as e:
            print(f"Fout bij invoegen verkoop (SalesID {row['SalesID']}): {e}")

    export_conn.commit()
    print(f"{len(final_df)} verkopen verwerkt in Fact_Sales")

move_feitsales(import_conn, export_conn)


C:\Users\ibrah\AppData\Local\Temp\ipykernel_15888\4020365802.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  header = pd.read_sql_query("""
C:\Users\ibrah\AppData\Local\Temp\ipykernel_15888\4020365802.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  detail = pd.read_sql_query("""


KeyError: 'SalesOrderID'

In [35]:
export_conn.close()
import_conn.close()